In [1]:
import torch
from transformers import GPT2LMHeadModel

In [2]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>') 
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [3]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [4]:
text = '잠을 못자게 되면'
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

잠을 못자게 되면 그걸로 끝이다.
그런데 이거야말로 정말 중요한 문제다.
이건 뭐냐 하면 우리가 흔히 말하는 '내부고발' 같은 거는 사실 우리 사회가 가지고 있는 가장 큰 문제점 중의 하나다.
우리 사회에서 내부 고발은 굉장히 많이 일어나고 있다.
그래서 이런 것들을 통해서 내부의 고발을 하는 것이 중요하다는 것을 알 수 있었다.
그리고 또 하나는 지금 현재 우리나라 사회에는 여러 가지 문제가 발생하고 있는데 특히나 최근에 일어난 사건들은 상당히 심각한 상황입니다.
그러니까 예를 들어서 어떤 사건이 터지면 바로 경찰이나 검찰에 가서 조사를 받고 그다음에 수사기관에 출석해서 조사받는 그런 상황이 벌어지기도 하고요.
또


In [5]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [6]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [7]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [9]:
ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
왜않되2.ipynb               왜않되.ipynb              model/
chatbot_data_30만.txt       감정분석kobert.ipynb      RealkoGpt_chatbot.ipynb
ChatBotData.csv             KoBert.ipynb              test_kogpt2.ipynb
data/                       kobert-wellness-chatbot/  Untitled1.ipynb
감정분석노가다옮기기.ipynb  koGpt_chatbot.ipynb       Untitled.ipynb


In [13]:
import urllib.request

Chatbot_Data = pd.read_csv("./data/dataset_33k.csv", encoding="cp949")
Chatbot_Data = Chatbot_Data[:33000]
Chatbot_Data.head()

,user,system,sentiment,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,system.1,sentiment.1
0,12시 땡!,하루가 또 가네요.,일상,NaN,NaN,NaN,NaN,NaN,NaN,NaN,하루가 또 가네요.,일상
1,1지망 학교 떨어졌어,위로해 드립니다.,일상,NaN,NaN,NaN,NaN,NaN,NaN,NaN,위로해 드립니다.,일상
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,일상,NaN,NaN,NaN,NaN,NaN,NaN,NaN,여행은 언제나 좋죠.,일상
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,일상,NaN,NaN,NaN,NaN,NaN,NaN,NaN,여행은 언제나 좋죠.,일상
4,PPL 심하네,눈살이 찌푸려지죠.,일상,NaN,NaN,NaN,NaN,NaN,NaN,NaN,눈살이 찌푸려지죠.,일상


In [25]:
Chatbot_Data.rename(columns={'system':'A'})
Chatbot_Data.rename(columns={'sentiment':'label'})

,user,system,label,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,system.1,sentiment.1
0,12시 땡!,하루가 또 가네요.,일상,NaN,NaN,NaN,NaN,NaN,NaN,NaN,하루가 또 가네요.,일상
1,1지망 학교 떨어졌어,위로해 드립니다.,일상,NaN,NaN,NaN,NaN,NaN,NaN,NaN,위로해 드립니다.,일상
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,일상,NaN,NaN,NaN,NaN,NaN,NaN,NaN,여행은 언제나 좋죠.,일상
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,일상,NaN,NaN,NaN,NaN,NaN,NaN,NaN,여행은 언제나 좋죠.,일상
4,PPL 심하네,눈살이 찌푸려지죠.,일상,NaN,NaN,NaN,NaN,NaN,NaN,NaN,눈살이 찌푸려지죠.,일상
...,...,...,...,...,...,...,...,...,...,...,...,...
32995,남들은 내가 유학 간다고 부러워하는데 난 너무 힘들어.,아주 간단한 일이라도 할 수 있을 것 같다는 생각이 들지 않나요?,무기력,NaN,NaN,NaN,NaN,NaN,NaN,NaN,아주 간단한 일이라도 할 수 있을 것 같다는 생각이 들지 않나요?,무기력
32996,근데 정말 애 키우기 너무 힘들다…,평소에 비해 말수가 적어졌나요?,무기력,NaN,NaN,NaN,NaN,NaN,NaN,NaN,평소에 비해 말수가 적어졌나요?,무기력
32997,첫째 아이 보는 것도 둘째 보는 것도 너무 버겁고 힘들어.,평소에는 아무렇지도 않던 일들이 귀찮게 느껴지나요?,무기력,NaN,NaN,NaN,NaN,NaN,NaN,NaN,평소에는 아무렇지도 않던 일들이 귀찮게 느껴지나요?,무기력
32998,첫째까진 괜찮았는데 둘째까지 같이 키우려니까 너무 힘들더라.,이전에 비해 매사에 의욕이 없나요?,무기력,NaN,NaN,NaN,NaN,NaN,NaN,NaN,이전에 비해 매사에 의욕이 없나요?,무기력


In [26]:
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=100):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["user"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["system"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [27]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [28]:
train_set = ChatbotDataset(Chatbot_Data, max_len=100)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)


In [29]:
print("start")
for batch_idx, samples in enumerate(train_dataloader):
    token_ids, mask, label = samples
    print("token_ids ====> ", token_ids)
    print("mask =====> ", mask)
    print("label =====> ", label)
print("end")

start
token_ids ====>  tensor([[    2, 14381,  7877,  ...,     3,     3,     3],
        [    2, 10305, 30478,  ...,     3,     3,     3],
        [    2, 22727,  9023,  ...,     3,     3,     3],
        ...,
        [    2,  9847, 10303,  ...,     3,     3,     3],
        [    2,  9863,  9179,  ...,     3,     3,     3],
        [    2, 10437, 10024,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2, 10437, 10024,  ...,     3,     3,     3],
        [    2,  9320,  9022,  ...,     3,     3,     3],
        [    2,  93

/tmp/ipykernel_881479/2495599640.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


token_ids ====>  tensor([[    2,  9320,  8022,  ...,     3,     3,     3],
        [    2,  9831,  9720,  ...,     3,     3,     3],
        [    2,  9183, 10389,  ...,     3,     3,     3],
        ...,
        [    2, 17369, 15866,  ...,     3,     3,     3],
        [    2,  9415,  6830,  ...,     3,     3,     3],
        [    2,  9105,  7512,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2,  9351, 27032,  ...,     3,     3,     3],
        [    2, 21466,  9193,  ...,     3,     3,     3],
        [    2, 15812,  7

token_ids ====>  tensor([[    2, 10428,  8703,  ...,     3,     3,     3],
        [    2, 10910, 10440,  ...,     3,     3,     3],
        [    2, 25689, 22031,  ...,     3,     3,     3],
        ...,
        [    2,  9078, 14826,  ...,     3,     3,     3],
        [    2,  9252,  7220,  ...,     3,     3,     3],
        [    2, 23971,  9022,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2, 15107,  9038,  ...,     3,     3,     3],
        [    2,  9320,  7991,  ...,     3,     3,     3],
        [    2,  9320,  8

token_ids ====>  tensor([[    2,  9769, 29205,  ...,     3,     3,     3],
        [    2, 10265,  9831,  ...,     3,     3,     3],
        [    2, 30478, 12748,  ...,     3,     3,     3],
        ...,
        [    2, 11594,  9199,  ...,     3,     3,     3],
        [    2, 42667, 11264,  ...,     3,     3,     3],
        [    2, 12687,  9267,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2,  9320, 31204,  ...,     3,     3,     3],
        [    2, 21277,  9518,  ...,     3,     3,     3],
        [    2, 11403, 12

...

token_ids ====>  tensor([[    2, 10099,  9420,  ...,     3,     3,     3],
        [    2, 17369, 15866,  ...,     3,     3,     3],
        [    2,  9183, 31416,  ...,     3,     3,     3],
        ...,
        [    2, 29205,  9723,  ...,     3,     3,     3],
        [    2, 10524,  9183,  ...,     3,     3,     3],
        [    2, 10185,  9193,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2,  9265,  7235,  ...,     3,     3,     3],
        [    2,  9060, 14701,  ...,     3,     3,     3],
        [    2,   739,  7

token_ids ====>  tensor([[    2, 12371, 25990,  ...,     3,     3,     3],
        [    2, 35704,  9420,  ...,     3,     3,     3],
        [    2, 24168, 33142,  ...,     3,     3,     3],
        ...,
        [    2, 12371, 45231,  ...,     3,     3,     3],
        [    2,  9350,  7065,  ...,     3,     3,     3],
        [    2,  9022,  9068,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2, 11307,  6855,  ...,     3,     3,     3],
        [    2,  9572,  9226,  ...,     3,     3,     3],
        [    2,  9103,  9

token_ids ====>  tensor([[    2, 12687,  9267,  ...,     3,     3,     3],
        [    2,  9022,  6855,  ...,     3,     3,     3],
        [    2,  9847,  9984,  ...,     3,     3,     3],
        ...,
        [    2,  9022, 23450,  ...,     3,     3,     3],
        [    2, 17483,  9153,  ...,     3,     3,     3],
        [    2,  9060, 14701,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2, 12079,  9586,  ...,     3,     3,     3],
        [    2,  9022,  6855,  ...,     3,     3,     3],
        [    2,  9063, 40

token_ids ====>  tensor([[    2, 15669,  7540,  ...,     3,     3,     3],
        [    2, 17582, 17484,  ...,     3,     3,     3],
        [    2, 36041, 43473,  ...,     3,     3,     3],
        ...,
        [    2, 10229, 12371,  ...,     3,     3,     3],
        [    2, 10377, 13413,  ...,     3,     3,     3],
        [    2, 11980,  9183,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2, 16962, 19716,  ...,     3,     3,     3],
        [    2, 10644, 12011,  ...,     3,     3,     3],
        [    2,  9320, 31

token_ids ====>  tensor([[    2, 14938, 20375,  ...,     3,     3,     3],
        [    2, 11928,  9272,  ...,     3,     3,     3],
        [    2,  9548,  6866,  ...,     3,     3,     3],
        ...,
        [    2, 22126,  9205,  ...,     3,     3,     3],
        [    2,  9320,  8084,  ...,     3,     3,     3],
        [    2, 40367, 11469,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2,  9265,  7162,  ...,     3,     3,     3],
        [    2, 23971, 12371,  ...,     3,     3,     3],
        [    2, 12079,  8

In [30]:
device = torch.device("cuda")
#device = torch.device("cpu")
train_set = ChatbotDataset(Chatbot_Data, max_len=100)
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [31]:
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [32]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 50
Sneg = -1e18

In [33]:
print("start")
for epoch in range(epoch):
    print(epoch)
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        token_ids = token_ids.to(device)
        mask = mask.to(device)
        label = label.to(device)
        out = model(token_ids)
        out = out.logits.to(device)
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2).to(device)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out)).to(device)
        loss = criterion(mask_out.transpose(2, 1), label).to(device)
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        optimizer.step()
print("end")

start
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
end


In [34]:
torch.save(model, f'./model_kogpt_chatbot_50epoch.pt')

In [35]:
torch.save(model, f'./model_kogpt_chatbot_50epoch.pth')

In [ ]:
with torch.no_grad():
    model.to('cuda')
    while True:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        sent = ""
        while True:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + sent + A_TKN + a)).unsqueeze(dim=0).to('cuda')
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))


user > 안녕
Chatbot > 안녕하세요
user > 반가워
Chatbot > 저도 반가워요
user > 오늘 날씨 좋다
Chatbot > 나들이 가보세요
user > 별일 없었어?
Chatbot > 그런 척 하는 걸 수도 있어요
user > 친구랑 다퉜는데 기분이 좀 안좋네
Chatbot > 싸우면서 정 들 거예요
user > 오늘 공부도 안되고 친구랑은 싸우고 좀 예민하고 기분이 많이 안좋아
Chatbot > 공부하면 더 많은 선택을 할 수 있죠
user > 공부도 안돼
Chatbot > 딴짓 중입니다
user > 피곤해
Chatbot > 충전하는 시간 그 자체로 소중합니다
user > 발표를 앞두고 있는데, 좀 긴장이 되네
Chatbot > 당신을 축하해요
user > 나 카카오 합격했어~!
Chatbot > 축하해요
